In [1]:
import pygame  # helps us make GUI games in python
import random  # help us define which direction the ball will start moving in

from pygame.transform import scale

# DQN. CNN reads in pixel data.
# reinforcement learning. trial and error.
# maximize action based on reward
# agent environment loop
# this is called Q Learning
# based on just game state. mapping of state to action is policy
# experience replay. learns from past policies


# frame rate per second
FPS = 60

# size of our window
WINDOW_WIDTH = 400
WINDOW_HEIGHT = 400

# size of our paddle
PADDLE_WIDTH = 10
PADDLE_HEIGHT = 60
# distance from the edge of the window
PADDLE_BUFFER = 10

# size of our ball
BALL_WIDTH = 10
BALL_HEIGHT = 10

# speeds of our paddle and ball
PADDLE_SPEED = 2
BALL_X_SPEED = 3
BALL_Y_SPEED = 2

# RGB colors for our paddle and ball
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)


# initialize our screen using width and height vars
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))


def drawBall(ballXPos, ballYPos):
    ball = pygame.Rect(ballXPos, ballYPos, BALL_WIDTH, BALL_HEIGHT)
    pygame.draw.rect(screen, WHITE, ball)


# Paddle 1 is our learning agent/us
# draw to the left of the screen
def drawPaddle1(paddle1YPos):
    paddle1 = pygame.Rect(PADDLE_BUFFER,
                          paddle1YPos,
                          PADDLE_WIDTH,
                          PADDLE_HEIGHT)
    pygame.draw.rect(screen, WHITE, paddle1)


# paddle 2 is the evil AI
# draw to the right of the screen
def drawPaddle2(paddle2YPos):
    paddle2 = pygame.Rect(WINDOW_WIDTH - PADDLE_BUFFER - PADDLE_WIDTH,
                          paddle2YPos,
                          PADDLE_WIDTH,
                          PADDLE_HEIGHT)
    pygame.draw.rect(screen, WHITE, paddle2)


# update the ball, using the paddle posistions the balls positions and the
# balls directions
def updateBall(paddle1YPos, paddle2YPos,
               ballXPos, ballYPos,
               ballXDirection, ballYDirection):

    # update the x and y position
    ballXPos = ballXPos + ballXDirection * BALL_X_SPEED
    ballYPos = ballYPos + ballYDirection * BALL_Y_SPEED
    score = 0

    # checks for a collision, if the ball hits the left side,
    # our learning agent
    if (ballXPos <= PADDLE_BUFFER + PADDLE_WIDTH and
        ballYPos + BALL_HEIGHT >= paddle1YPos and
        ballYPos - BALL_HEIGHT <= paddle1YPos + PADDLE_HEIGHT):
        # switches directions
        ballXDirection = 1
    # past it
    elif (ballXPos <= 0):
        # negative score
        ballXDirection = 1
        score = -1
        return [score, paddle1YPos, paddle2YPos,
                ballXPos, ballYPos,
                ballXDirection, ballYDirection]

    # check if hits the other side
    if (ballXPos >= WINDOW_WIDTH - PADDLE_WIDTH - PADDLE_BUFFER and
        ballYPos + BALL_HEIGHT >= paddle2YPos and
        ballYPos - BALL_HEIGHT <= paddle2YPos + PADDLE_HEIGHT):
        # switch directions
        ballXDirection = -1
    # past it
    elif (ballXPos >= WINDOW_WIDTH - BALL_WIDTH):
        # positive score
        ballXDirection = -1
        score = 1
        return [score, paddle1YPos, paddle2YPos,
                ballXPos, ballYPos,
                ballXDirection, ballYDirection]

    # if it hits the top, move down
    if (ballYPos <= 0):
        ballYPos = 0
        ballYDirection = 1
    # if it hits the bottom, move up
    elif (ballYPos >= WINDOW_HEIGHT - BALL_HEIGHT):
        ballYPos = WINDOW_HEIGHT - BALL_HEIGHT
        ballYDirection = -1
    return [score, paddle1YPos, paddle2YPos,
            ballXPos, ballYPos,
            ballXDirection, ballYDirection]


# update our paddle position
# this is controlled by the action input
def updatePaddle1(action, paddle1YPos):
    # if move up
    if (action[1] == 1):
        paddle1YPos = paddle1YPos - PADDLE_SPEED
    # if move down
    if (action[2] == 1):
        paddle1YPos = paddle1YPos + PADDLE_SPEED

    # don't let it move off the screen
    if (paddle1YPos < 0):
        paddle1YPos = 0
    if (paddle1YPos > WINDOW_HEIGHT - PADDLE_HEIGHT):
        paddle1YPos = WINDOW_HEIGHT - PADDLE_HEIGHT
    return paddle1YPos

# update evil AI paddle position
# this is controlled by the position of the ball
def updatePaddle2(paddle2YPos, ballYPos):
    # move down if ball is in upper half
    if (paddle2YPos + PADDLE_HEIGHT / 2 < ballYPos + BALL_HEIGHT / 2):
        paddle2YPos = paddle2YPos + PADDLE_SPEED
    # move up if ball is in lower half
    if (paddle2YPos + PADDLE_HEIGHT / 2 > ballYPos + BALL_HEIGHT / 2):
        paddle2YPos = paddle2YPos - PADDLE_SPEED
    # don't let it hit top
    if (paddle2YPos < 0):
        paddle2YPos = 0
    # dont let it hit bottom
    if (paddle2YPos > WINDOW_HEIGHT - PADDLE_HEIGHT):
        paddle2YPos = WINDOW_HEIGHT - PADDLE_HEIGHT
    return paddle2YPos


# game class
class PongGame:
    def __init__(self):
        # random number for initial direction of ball
        num = random.randint(0, 9)
        # keep score
        self.tally = 0
        # initialie positions of paddle
        self.paddle1YPos = WINDOW_HEIGHT / 2 - PADDLE_HEIGHT / 2
        self.paddle2YPos = WINDOW_HEIGHT / 2 - PADDLE_HEIGHT / 2
        # and ball direction
        self.ballXDirection = 1
        self.ballYDirection = 1
        # starting point
        self.ballXPos = WINDOW_WIDTH / 2 - BALL_WIDTH / 2

        # randomly decide where the ball will move
        if(0 < num < 3):
            self.ballXDirection = 1
            self.ballYDirection = 1
        if (3 <= num < 5):
            self.ballXDirection = -1
            self.ballYDirection = 1
        if (5 <= num < 8):
            self.ballXDirection = 1
            self.ballYDirection = -1
        if (8 <= num < 10):
            self.ballXDirection = -1
            self.ballYDirection = -1
        # new random number
        num = random.randint(0, 9)
        # where it will start, y part
        self.ballYPos = num * (WINDOW_HEIGHT - BALL_HEIGHT) / 9
        # scaled surface
        self.scaled_surface = pygame.Surface((84, 84), depth=32)


    def getPresentFrame(self):
        # for each frame, calls the event queue, like if the main window needs
        # to be repainted
        pygame.event.pump()
        # make the background black
        screen.fill(BLACK)
        # draw our paddles
        drawPaddle1(self.paddle1YPos)
        drawPaddle2(self.paddle2YPos)
        # draw our ball
        drawBall(self.ballXPos, self.ballYPos)
        # updates the window
        pygame.display.flip()

        # copies the pixels from our surface to a 3D array. we'll use this for
        # RL
        pygame.transform.scale(pygame.display.get_surface(), (84, 84), self.scaled_surface)
        image_data = pygame.surfarray.array2d(self.scaled_surface)
        # return our surface data
        return image_data


    def getNextFrame(self, action):
        pygame.event.pump()
        score = 0
        screen.fill(BLACK)
        
        # update our paddle
        self.paddle1YPos = updatePaddle1(action, self.paddle1YPos)
        drawPaddle1(self.paddle1YPos)

        # update evil AI paddle
        self.paddle2YPos = updatePaddle2(self.paddle2YPos, self.ballYPos)
        drawPaddle2(self.paddle2YPos)
        
        # update our vars by updating ball position
        [score, self.paddle1YPos, self.paddle2YPos, self.ballXPos,
         self.ballYPos, self.ballXDirection, self.ballYDirection] = updateBall(self.paddle1YPos, self.paddle2YPos, self.ballXPos,
                                                                               self.ballYPos, self.ballXDirection, self.ballYDirection)  # draw the ball
        drawBall(self.ballXPos, self.ballYPos)
        
        # get the surface data
        # copies the pixels from our surface to a 3D array. we'll use this for
        # RL
        pygame.transform.scale(pygame.display.get_surface(), (84, 84), self.scaled_surface)
        image_data = pygame.surfarray.array2d(self.scaled_surface)
        # return our surface data
        # update the window
        pygame.display.flip()
        # record the total score
        self.tally = self.tally + score
        print("Tally is " + str(self.tally))
        # return the score and the surface data
        return [score, image_data]

pygame 1.9.5
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import tensorflow as tf
#import pong  # our class
import numpy as np  # math
import random  # random
# queue data structure. fast appends. and pops. replay memory
from collections import deque


# hyper params
ACTIONS = 3  # up, down, stay
# define our learning rate
GAMMA = 0.99
# for updating our gradient or training over time
INITIAL_EPSILON = 1.0
FINAL_EPSILON = 0.05
# how many frames to anneal epsilon
EXPLORE = 500000
OBSERVE = 50000
# store our experiences, the size of it
REPLAY_MEMORY = 500000
# batch size to train on
BATCH = 100
# input image size in pixels
INPUT_SIZE = 84

# create tensorflow graph
def createGraph():

    # CNN
    # creates an empty tensor with all elements set to zero with a shape
    W_conv1 = tf.Variable(tf.random_normal([8, 8, 4, 32]))
    b_conv1 = tf.Variable(tf.random_normal([32]))

    W_conv2 = tf.Variable(tf.random_normal([4, 4, 32, 64]))
    b_conv2 = tf.Variable(tf.random_normal([64]))

    W_conv3 = tf.Variable(tf.random_normal([3, 3, 64, 64]))
    b_conv3 = tf.Variable(tf.random_normal([64]))

    W_fc4 = tf.Variable(tf.random_normal([7 * 7 * 64, 784]))  # image size 7x7 due to convolutions
    b_fc4 = tf.Variable(tf.random_normal([784]))

    W_fc5 = tf.Variable(tf.random_normal([784, ACTIONS]))
    b_fc5 = tf.Variable(tf.random_normal([ACTIONS]))

    # input for pixel data
    s = tf.placeholder("float", [None, INPUT_SIZE, INPUT_SIZE, 4])

    # Computes rectified linear unit activation fucntion on  a 2-D convolution
    # given 4-D input and filter tensors. and
    conv1 = tf.nn.relu(
        tf.nn.conv2d(s, W_conv1, strides=[1, 4, 4, 1], padding="VALID") + b_conv1)

    conv2 = tf.nn.relu(
        tf.nn.conv2d(conv1, W_conv2, strides=[1, 2, 2, 1], padding="VALID") + b_conv2)

    conv3 = tf.nn.relu(
        tf.nn.conv2d(conv2, W_conv3, strides=[1, 1, 1, 1], padding="VALID") + b_conv3)

    conv3_flat = tf.reshape(conv3, [-1, 7 * 7 * 64])

    fc4 = tf.nn.relu(tf.matmul(conv3_flat, W_fc4) + b_fc4)

    fc5 = tf.matmul(fc4, W_fc5) + b_fc5

    # return input and output to the network
    return s, fc5



# deep q network. feed in pixel data to graph session
def trainGraph(inp, out, sess):

    # to calculate the argmax, we multiply the predicted output with a vector
    # with one value 1 and rest as 0
    argmax = tf.placeholder("float", [None, ACTIONS])
    gt = tf.placeholder("float", [None])  # ground truth

    # action
    action = tf.reduce_sum(tf.multiply(out, argmax), reduction_indices=1)
    # cost function we will reduce through backpropagation
    cost = tf.reduce_mean(tf.square(action - gt))
    # optimization function to reduce our minimize our cost function
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # initialize our game
    game = PongGame()

    # create a queue for experience replay to store policies
    D = deque()

    # intial frame
    frame = game.getPresentFrame()
    # convert rgb to gray scale for processing

    # stack frames, that is our input tensor
    inp_t = np.stack((frame, frame, frame, frame), axis=2)

    # saver
    saver = tf.train.Saver()

    sess.run(tf.global_variables_initializer())

    t = 0
    epsilon = INITIAL_EPSILON

    # training time
    while(1):
        # output tensor
        out_t = out.eval(feed_dict={inp: [inp_t]})[0]
        # argmax function
        argmax_t = np.zeros([ACTIONS])

        # random action with prob epsilon
        if(random.random() <= epsilon):
            maxIndex = random.randrange(ACTIONS)
        # predicted action with prob (1 - epsilon)
        else:
            maxIndex = np.argmax(out_t)
        argmax_t[maxIndex] = 1

        if epsilon > FINAL_EPSILON:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # reward tensor if score is positive
        reward_t, frame = game.getNextFrame(argmax_t)

        frame = np.reshape(frame, (INPUT_SIZE, INPUT_SIZE, 1))
        
        # new input tensor
        inp_t1 = np.append(frame, inp_t[:, :, 0:3], axis=2)

        # add our input tensor, argmax tensor, reward and updated input tensor
        # to stack of experiences
        D.append((inp_t, argmax_t, reward_t, inp_t1))

        # if we run out of replay memory, make room
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        # training iteration
        if t > OBSERVE:

            # get values from our replay memory
            minibatch = random.sample(D, BATCH)

            inp_batch = [d[0] for d in minibatch]
            argmax_batch = [d[1] for d in minibatch]
            reward_batch = [d[2] for d in minibatch]
            inp_t1_batch = [d[3] for d in minibatch]

            gt_batch = []
            out_batch = out.eval(feed_dict={inp: inp_t1_batch})

            # add values to our batch
            for i in range(0, len(minibatch)):
                gt_batch.append(reward_batch[i] + GAMMA * np.max(out_batch[i]))

            # train on that
            train_step.run(feed_dict={
                           gt: gt_batch,
                           argmax: argmax_batch,
                           inp: inp_batch
                           })

        # update our input tensor the the next frame
        inp_t = inp_t1
        t = t+1

        # print our where wer are after saving where we are
        if t % 10000 == 0:
            saver.save(sess, './' + 'pong' + '-dqn', global_step=t)

        print("TIMESTEP", t, "/ EPSILON", epsilon, "/ ACTION", maxIndex,
              "/ REWARD", reward_t, "/ Q_MAX %e" % np.max(out_t))


def main():
    # create session
    sess = tf.InteractiveSession()
    # input layer and output layer by creating graph
    inp, out = createGraph()
    # train our graph on input and output with session variables
    trainGraph(inp, out, sess)

if __name__ == "__main__":
    main()

Tally is 0
TIMESTEP 1 / EPSILON 0.9999981 / ACTION 0 / REWARD 0 / Q_MAX -1.096599e+12
Tally is 0
TIMESTEP 2 / EPSILON 0.9999962 / ACTION 0 / REWARD 0 / Q_MAX -6.037113e+11
Tally is 0
TIMESTEP 3 / EPSILON 0.9999943 / ACTION 2 / REWARD 0 / Q_MAX -2.538163e+11
Tally is 0
TIMESTEP 4 / EPSILON 0.9999924 / ACTION 1 / REWARD 0 / Q_MAX 2.830074e+11
Tally is 0
TIMESTEP 5 / EPSILON 0.9999905 / ACTION 1 / REWARD 0 / Q_MAX -1.633174e+11
Tally is 0
TIMESTEP 6 / EPSILON 0.9999886 / ACTION 2 / REWARD 0 / Q_MAX 1.548097e+12
Tally is 0
TIMESTEP 7 / EPSILON 0.9999867 / ACTION 2 / REWARD 0 / Q_MAX 1.111687e+12
Tally is 0
TIMESTEP 8 / EPSILON 0.9999848 / ACTION 1 / REWARD 0 / Q_MAX 1.584131e+12
Tally is 0
TIMESTEP 9 / EPSILON 0.9999829 / ACTION 2 / REWARD 0 / Q_MAX 1.236410e+12
Tally is 0
TIMESTEP 10 / EPSILON 0.999981 / ACTION 1 / REWARD 0 / Q_MAX -1.052681e+12
Tally is 0
TIMESTEP 11 / EPSILON 0.9999791 / ACTION 0 / REWARD 0 / Q_MAX -1.912115e+12
Tally is 0
TIMESTEP 12 / EPSILON 0.9999772 / ACTION 0 / RE

TIMESTEP 118 / EPSILON 0.9997758000000001 / ACTION 2 / REWARD 0 / Q_MAX 7.418386e+11
Tally is -1
TIMESTEP 119 / EPSILON 0.9997739000000001 / ACTION 2 / REWARD 0 / Q_MAX 1.657658e+12
Tally is -1
TIMESTEP 120 / EPSILON 0.9997720000000001 / ACTION 2 / REWARD 0 / Q_MAX 6.696868e+11
Tally is -1
TIMESTEP 121 / EPSILON 0.9997701000000001 / ACTION 1 / REWARD 0 / Q_MAX 1.767578e+12
Tally is -1
TIMESTEP 122 / EPSILON 0.9997682000000001 / ACTION 0 / REWARD 0 / Q_MAX 2.705456e+11
Tally is -1
TIMESTEP 123 / EPSILON 0.9997663000000001 / ACTION 1 / REWARD 0 / Q_MAX 6.074151e+11
Tally is -1
TIMESTEP 124 / EPSILON 0.9997644000000001 / ACTION 2 / REWARD 0 / Q_MAX 4.862117e+11
Tally is -1
TIMESTEP 125 / EPSILON 0.9997625000000001 / ACTION 2 / REWARD 0 / Q_MAX 2.618701e+11
Tally is -1
TIMESTEP 126 / EPSILON 0.9997606000000001 / ACTION 1 / REWARD 0 / Q_MAX 1.764291e+12
Tally is -1
TIMESTEP 127 / EPSILON 0.9997587000000001 / ACTION 2 / REWARD 0 / Q_MAX 1.026607e+12
Tally is -1
TIMESTEP 128 / EPSILON 0.99975

Tally is -1
TIMESTEP 214 / EPSILON 0.9995934000000002 / ACTION 0 / REWARD 0 / Q_MAX -7.318311e+11
Tally is -1
TIMESTEP 215 / EPSILON 0.9995915000000002 / ACTION 0 / REWARD 0 / Q_MAX -1.840837e+12
Tally is -1
TIMESTEP 216 / EPSILON 0.9995896000000002 / ACTION 0 / REWARD 0 / Q_MAX -2.715413e+12
Tally is -1
TIMESTEP 217 / EPSILON 0.9995877000000002 / ACTION 0 / REWARD 0 / Q_MAX -1.941635e+12
Tally is -1
TIMESTEP 218 / EPSILON 0.9995858000000002 / ACTION 2 / REWARD 0 / Q_MAX -1.707902e+12
Tally is -1
TIMESTEP 219 / EPSILON 0.9995839000000002 / ACTION 0 / REWARD 0 / Q_MAX -1.751903e+12
Tally is -1
TIMESTEP 220 / EPSILON 0.9995820000000002 / ACTION 0 / REWARD 0 / Q_MAX -4.219571e+11
Tally is -1
TIMESTEP 221 / EPSILON 0.9995801000000002 / ACTION 1 / REWARD 0 / Q_MAX -2.784673e+12
Tally is -1
TIMESTEP 222 / EPSILON 0.9995782000000002 / ACTION 1 / REWARD 0 / Q_MAX 2.039669e+10
Tally is -1
TIMESTEP 223 / EPSILON 0.9995763000000002 / ACTION 2 / REWARD 0 / Q_MAX 6.233755e+11
Tally is -1
TIMESTEP 2

TIMESTEP 311 / EPSILON 0.9994091000000003 / ACTION 2 / REWARD 0 / Q_MAX 4.595987e+11
Tally is -1
TIMESTEP 312 / EPSILON 0.9994072000000003 / ACTION 2 / REWARD 0 / Q_MAX 6.575646e+11
Tally is -1
TIMESTEP 313 / EPSILON 0.9994053000000003 / ACTION 1 / REWARD 0 / Q_MAX 7.634502e+11
Tally is -1
TIMESTEP 314 / EPSILON 0.9994034000000003 / ACTION 2 / REWARD 0 / Q_MAX 2.543465e+11
Tally is -1
TIMESTEP 315 / EPSILON 0.9994015000000003 / ACTION 0 / REWARD 0 / Q_MAX 2.771326e+11
Tally is -1
TIMESTEP 316 / EPSILON 0.9993996000000003 / ACTION 1 / REWARD 0 / Q_MAX 3.084241e+11
Tally is -1
TIMESTEP 317 / EPSILON 0.9993977000000003 / ACTION 1 / REWARD 0 / Q_MAX 2.281467e+11
Tally is -1
TIMESTEP 318 / EPSILON 0.9993958000000003 / ACTION 1 / REWARD 0 / Q_MAX 5.366816e+11
Tally is -2
TIMESTEP 319 / EPSILON 0.9993939000000003 / ACTION 0 / REWARD -1 / Q_MAX 2.779084e+11
Tally is -2
TIMESTEP 320 / EPSILON 0.9993920000000003 / ACTION 0 / REWARD 0 / Q_MAX 6.840377e+10
Tally is -2
TIMESTEP 321 / EPSILON 0.9993

TIMESTEP 400 / EPSILON 0.9992400000000004 / ACTION 1 / REWARD 0 / Q_MAX 1.731099e+12
Tally is -2
TIMESTEP 401 / EPSILON 0.9992381000000004 / ACTION 2 / REWARD 0 / Q_MAX 1.960690e+12
Tally is -2
TIMESTEP 402 / EPSILON 0.9992362000000004 / ACTION 0 / REWARD 0 / Q_MAX 8.411783e+11
Tally is -2
TIMESTEP 403 / EPSILON 0.9992343000000004 / ACTION 1 / REWARD 0 / Q_MAX 6.496329e+11
Tally is -2
TIMESTEP 404 / EPSILON 0.9992324000000004 / ACTION 2 / REWARD 0 / Q_MAX 2.485543e+12
Tally is -2
TIMESTEP 405 / EPSILON 0.9992305000000004 / ACTION 2 / REWARD 0 / Q_MAX 2.690676e+12
Tally is -2
TIMESTEP 406 / EPSILON 0.9992286000000004 / ACTION 0 / REWARD 0 / Q_MAX 5.286725e+11
Tally is -2
TIMESTEP 407 / EPSILON 0.9992267000000004 / ACTION 0 / REWARD 0 / Q_MAX 7.848024e+11
Tally is -2
TIMESTEP 408 / EPSILON 0.9992248000000004 / ACTION 0 / REWARD 0 / Q_MAX 7.962672e+11
Tally is -2
TIMESTEP 409 / EPSILON 0.9992229000000004 / ACTION 0 / REWARD 0 / Q_MAX 1.989923e+12
Tally is -2
TIMESTEP 410 / EPSILON 0.99922

...

Tally is -72
TIMESTEP 23362 / EPSILON 0.9556122000000205 / ACTION 2 / REWARD 0 / Q_MAX 2.065452e+12
Tally is -72
TIMESTEP 23363 / EPSILON 0.9556103000000205 / ACTION 1 / REWARD 0 / Q_MAX 1.424537e+12
Tally is -72
TIMESTEP 23364 / EPSILON 0.9556084000000205 / ACTION 0 / REWARD 0 / Q_MAX 1.588125e+12
Tally is -72
TIMESTEP 23365 / EPSILON 0.9556065000000205 / ACTION 1 / REWARD 0 / Q_MAX 9.819368e+11
Tally is -72
TIMESTEP 23366 / EPSILON 0.9556046000000205 / ACTION 2 / REWARD 0 / Q_MAX 1.704612e+12
Tally is -72
TIMESTEP 23367 / EPSILON 0.9556027000000205 / ACTION 2 / REWARD 0 / Q_MAX 1.759777e+12
Tally is -72
TIMESTEP 23368 / EPSILON 0.9556008000000205 / ACTION 1 / REWARD 0 / Q_MAX 2.702631e+12
Tally is -72
TIMESTEP 23369 / EPSILON 0.9555989000000205 / ACTION 2 / REWARD 0 / Q_MAX 2.314308e+12
Tally is -72
TIMESTEP 23370 / EPSILON 0.9555970000000205 / ACTION 2 / REWARD 0 / Q_MAX 2.577606e+12
Tally is -72
TIMESTEP 23371 / EPSILON 0.9555951000000205 / ACTION 1 / REWARD 0 / Q_MAX 3.920580e+12


Tally is -73
TIMESTEP 23462 / EPSILON 0.9554222000000205 / ACTION 1 / REWARD 0 / Q_MAX 2.369620e+12
Tally is -73
TIMESTEP 23463 / EPSILON 0.9554203000000205 / ACTION 0 / REWARD 0 / Q_MAX 2.731727e+12
Tally is -73
TIMESTEP 23464 / EPSILON 0.9554184000000205 / ACTION 2 / REWARD 0 / Q_MAX 2.178220e+12
Tally is -73
TIMESTEP 23465 / EPSILON 0.9554165000000205 / ACTION 0 / REWARD 0 / Q_MAX 2.422440e+12
Tally is -73
TIMESTEP 23466 / EPSILON 0.9554146000000205 / ACTION 1 / REWARD 0 / Q_MAX 2.178169e+12
Tally is -73
TIMESTEP 23467 / EPSILON 0.9554127000000205 / ACTION 0 / REWARD 0 / Q_MAX 8.134791e+11
Tally is -73
TIMESTEP 23468 / EPSILON 0.9554108000000205 / ACTION 2 / REWARD 0 / Q_MAX 7.887298e+11
Tally is -73
TIMESTEP 23469 / EPSILON 0.9554089000000205 / ACTION 0 / REWARD 0 / Q_MAX 8.815851e+11
Tally is -73
TIMESTEP 23470 / EPSILON 0.9554070000000205 / ACTION 1 / REWARD 0 / Q_MAX 3.560462e+12
Tally is -73
TIMESTEP 23471 / EPSILON 0.9554051000000205 / ACTION 2 / REWARD 0 / Q_MAX 4.008401e+12


Tally is -73
TIMESTEP 23560 / EPSILON 0.9552360000000206 / ACTION 1 / REWARD 0 / Q_MAX 1.306791e+12
Tally is -73
TIMESTEP 23561 / EPSILON 0.9552341000000206 / ACTION 2 / REWARD 0 / Q_MAX 1.122673e+12
Tally is -73
TIMESTEP 23562 / EPSILON 0.9552322000000206 / ACTION 1 / REWARD 0 / Q_MAX 8.958318e+11
Tally is -73
TIMESTEP 23563 / EPSILON 0.9552303000000206 / ACTION 0 / REWARD 0 / Q_MAX 1.080822e+12
Tally is -73
TIMESTEP 23564 / EPSILON 0.9552284000000206 / ACTION 0 / REWARD 0 / Q_MAX 1.219619e+12
Tally is -73
TIMESTEP 23565 / EPSILON 0.9552265000000206 / ACTION 1 / REWARD 0 / Q_MAX 2.609031e+11
Tally is -73
TIMESTEP 23566 / EPSILON 0.9552246000000206 / ACTION 0 / REWARD 0 / Q_MAX -2.910587e+11
Tally is -73
TIMESTEP 23567 / EPSILON 0.9552227000000206 / ACTION 2 / REWARD 0 / Q_MAX -2.399515e+10
Tally is -73
TIMESTEP 23568 / EPSILON 0.9552208000000206 / ACTION 2 / REWARD 0 / Q_MAX 8.336913e+11
Tally is -73
TIMESTEP 23569 / EPSILON 0.9552189000000206 / ACTION 1 / REWARD 0 / Q_MAX 3.906502e+1

TIMESTEP 23658 / EPSILON 0.9550498000000207 / ACTION 0 / REWARD 0 / Q_MAX 6.153456e+12
Tally is -73
TIMESTEP 23659 / EPSILON 0.9550479000000207 / ACTION 2 / REWARD 0 / Q_MAX 5.463896e+12
Tally is -73
TIMESTEP 23660 / EPSILON 0.9550460000000207 / ACTION 0 / REWARD 0 / Q_MAX 5.874049e+12
Tally is -73
TIMESTEP 23661 / EPSILON 0.9550441000000207 / ACTION 2 / REWARD 0 / Q_MAX 3.912807e+12
Tally is -73
TIMESTEP 23662 / EPSILON 0.9550422000000207 / ACTION 2 / REWARD 0 / Q_MAX 2.455164e+12
Tally is -73
TIMESTEP 23663 / EPSILON 0.9550403000000207 / ACTION 0 / REWARD 0 / Q_MAX 2.863596e+12
Tally is -73
TIMESTEP 23664 / EPSILON 0.9550384000000207 / ACTION 2 / REWARD 0 / Q_MAX 1.596847e+12
Tally is -73
TIMESTEP 23665 / EPSILON 0.9550365000000207 / ACTION 1 / REWARD 0 / Q_MAX 1.966284e+12
Tally is -73
TIMESTEP 23666 / EPSILON 0.9550346000000207 / ACTION 1 / REWARD 0 / Q_MAX 1.668461e+12
Tally is -73
TIMESTEP 23667 / EPSILON 0.9550327000000207 / ACTION 0 / REWARD 0 / Q_MAX 3.827844e+12
Tally is -73


KeyboardInterrupt: 